# 懶人投資包-專為白痴設計的交易策略

In [ ]:
import tech
import pickle
import numpy as np
import pandas as pd
import math

In [ ]:
with open('all_stockdata','rb') as f:
    data=pickle.load(file=f)

In [ ]:
data_df=pd.concat(data.values()).reset_index().sort_values('Date').reset_index(drop=True)
datelist=sorted(list(set(data_df['Date'])))

# 問卷

In [ ]:
ans1 = input("1.年齡 \n(1)69歲以上/20歲以下 (2)66-68歲 (3)56-65歲 (4)41-55歲 (5)20-40歲\n"+"請輸入您的答案")
print("您選擇  "+ ans1 + "\n")
ans2 = input("2.金融理財知識：您對金融市場和投資的認識？\n(1) 我對金融市場一無所知，但有興趣進一步瞭解。\n(2) 我對金融市場只有一些基本知識，例如股票與基金的分別。\n(3) 我瞭解基本知識，並明白分散投資及資產配置的重要性，並作出分散投資。\n(4) 我對金融商品及其投資風險有進一步的認識，並明白影響股票和債券價格的因素。\n(5) 我非常熟悉大部份金融產品(包括債券、股票、認股權證、期權及期貨)，並明白影響這些金融產品的風險和表 現的各項因素。\n"+"請輸入您的答案")
print("您選擇  "+ ans2 + "\n")
ans3 = input("3.金融投資商品的投資經驗, 包括期貨、股票、債券、基金、投資型保單、衍生性投資工具...等\n(1) 1 年（含）以下 (2) 1 年- 3 年（含） (3) 3 年- 6 年（含） (4) 6 年- 10 年（含） (5) 10 年以上\n" +"請輸入您的答案")
print("您選擇  "+ ans3 + "\n")
ans4 = input("4.可承受的波動程度：一般而言，您可接受何種程度的價格波動？\n(1) 無法接受波動 (2) 可接受最高 5% 的波動 (3) 可接受最高 15% 的波動 (4) 可接受最高 30% 的波動 (5) 可接受 30% 以上的波動\n"+"請輸入您的答案")
print("您選擇  "+ ans4 + "\n")
ans5 = input("5.投資年間：一般而言，當您投資的金融商品有波動時，您可接受的持有期間？\n(1) 3 個月以下 (2) 3 個月- 6 個月（含） (3) 6 個月- 1 年（含） (4) 1 年- 3 年（含） (5) 3 年以上\n"+"請輸入您的答案")
print("您選擇  "+ ans5 + "\n")
ans6 = input("6.財務狀況：在一般情況下，您的家庭月收入約有多少比例可以用於投資或儲蓄？\n(1) 無 (2) 介於 0% - 10%（含） (3) 介於 10% - 30%（含）(4) 介於 30% - 60%（含） (5) 60% 以上\n"+"請輸入您的答案")
print("您選擇  "+ ans6 + "\n")
ans7 = input("7.資產配置：目前投資的資產中（不包括自用房地產），約有多少比例是持有金融投資商品？\n(1) 無 (2) > 0% - 10% (3) > 10% - 25% (4) > 25% - 50% (5) > 50%\n"+"請輸入您的答案")
print("您選擇  "+ ans7 + "\n")
ans8 = input("8.金融投資商品的交易頻率？\n(1) 一年以上 (2) 半年 (3) 每季 (4) 每月 (5) 每週\n"+"請輸入您的答案")
print("您選擇  "+ ans8 + "\n")
ans9 = input("9.投資目標：下列哪一項最能描述您的投資目標？\n(1) 首要目標是能維持本金，即使投資回報率可能非常低。\n(2) 追求穩定的定期收益，例如股息或是債券配息，即使存在資本虧損的風險。\n(3) 在穩定的資本成長與定期收益之間追求平衡的投資回報率。\n(4) 願意承擔較高的風險，追求資本增值，以累積資金。\n(5) 願意承擔更高的風險，以儘量提高資本增值。\n"+"請輸入您的答案")
print("您選擇  "+ ans9 + "\n")
ans10 = input("10若非預期的事件發生時，請問您的備用金相當於您幾個月的家庭開銷？\n(1) 無備用金儲蓄* (2) 3 個月以下 (3) 介於 3 個月到 6 個月 (4) 介於 6 個月到 9 個月 (5) 超過 9 個月 \n"+"請輸入您的答案")
print("您選擇  "+ ans10 + "\n")

#計算分數
scores = [ans1, ans2, ans3, ans4, ans5, ans6, ans7, ans8, ans9, ans10]
scores = map(int, scores)

all = []
for i in scores:
    all.append(i)

s = 0
for n in all:
    s = s+n
print("您的分數是", s ,"分")

# 判斷風險類型

In [ ]:
trade_table={}
if s <= 15:
    print ("您是保守型")
    # 計算各支股票的回測結果

    strategies = [tech.KD_way]
    #for i in range(1,5):
    for symbol in data:
        for strategy in strategies:
            try:
                tech.apply_strategy(strategy, data[symbol])
                if np.all(data[symbol]['signals']==0):
                    print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                elif np.any(data[symbol]['signals']!=0):
                    trade_table["{0}".format(symbol)] = data[symbol].loc[data[symbol]['signals'] != 0]
                    continue
            except Exception as e:
                print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)
    #去除沒有成對的交易
    for symbol in trade_table:
        if trade_table[symbol].iloc[trade_table[symbol].index.size-1,9] == 1:
            trade_table[symbol].drop(trade_table[symbol].index[trade_table[symbol]['signals'].size-1],inplace=True)
    
    #取出dataframe
    Total=pd.concat(trade_table.values()).reset_index()
    
    #整理交易明細表
    datelist=sorted(list(set(Total['Date'])))

    buydata = Total.loc[Total['signals'] == 1]
    buydata.columns = ['Buy_Date','Open','High','Low','buyPrice','Adj Close','Volume','symbol','K','D','signals','positions']
    buydata=buydata.reset_index()
    buydata.drop(['index','Open','High','Low','Adj Close','K','D','positions'],axis=1,inplace=True)

    selldata = Total.loc[Total['signals'] == -1]
    selldata.columns = ['Sell_Date','Open','High','Low','sellPrice','Adj Close','Volume','symbol','K','D','signals','positions']
    selldata=selldata.reset_index()
    selldata.drop(['index','Open','High','Low','Adj Close','symbol','K','D'],axis=1,inplace=True)

    result=pd.concat([buydata,selldata], axis=1)
    result=result.sort_values('Buy_Date').reset_index(drop=True)
        
elif s>=16 and s<=30:
    print("您是穩健型")
    # 計算各支股票的回測結果

    strategies = [tech.MACD]
    #for i in range(1,5):
    for symbol in data:
        for strategy in strategies:
            try:
                tech.apply_strategy(strategy, data[symbol])
                if np.all(data[symbol]['signals']==0):
                    print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                elif np.any(data[symbol]['signals']!=0):
                    #tradetable = data[symbol].loc[data[symbol]['signals'] != 0]
                    trade_table["{0}".format(symbol)] = data[symbol].loc[data[symbol]['signals'] != 0]
                    continue
            except Exception as e:
                print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)
    #去除沒有成對的交易
    for symbol in trade_table:
        if trade_table[symbol].iloc[trade_table[symbol].index.size-1,10] == 1:
            trade_table[symbol].drop(trade_table[symbol].index[trade_table[symbol]['signals'].size-1],inplace=True)
    
    #取出dataframe
    Total=pd.concat(trade_table.values()).reset_index()
    
    #整理交易明細表
    datelist=sorted(list(set(Total['Date'])))

    buydata = Total.loc[Total['signals'] == 1]
    buydata.columns = ['Buy_Date','Open','High','Low','buyPrice','Adj Close','Volume','symbol','DIF','DEA','MACD',
                       'signals','positions']
    buydata=buydata.reset_index()
    buydata.drop(['index','Open','High','Low','Adj Close','DIF','DEA','MACD','positions'],axis=1,inplace=True)

    selldata = Total.loc[Total['signals'] == -1]
    selldata.columns = ['Sell_Date','Open','High','Low','sellPrice','Adj Close','Volume','symbol','DIF','DEA','MACD',
                        'signals','positions']
    selldata=selldata.reset_index()
    selldata.drop(['index','Open','High','Low','Adj Close','symbol','DIF','DEA','MACD'],axis=1,inplace=True)

    result=pd.concat([buydata,selldata], axis=1)
    result=result.sort_values('Buy_Date').reset_index(drop=True)
        
else:
    print("您是積極型")
    # 計算各支股票的回測結果

    strategies = [tech.BBands_strategy]
    #for i in range(1,5):
    for symbol in data:
        for strategy in strategies:
            try:
                tech.apply_strategy(strategy, data[symbol])
                if np.all(data[symbol]['signals']==0):
                    print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                elif np.any(data[symbol]['signals']!=0):
                    #tradetable = data[symbol].loc[data[symbol]['signals'] != 0]
                    trade_table["{0}".format(symbol)] = data[symbol].loc[data[symbol]['signals'] != 0]
                    continue
            except Exception as e:
                print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)
    #去除沒有成對的交易
    for symbol in trade_table:
        if trade_table[symbol].iloc[trade_table[symbol].index.size-1,12] == 1:
            trade_table[symbol].drop(trade_table[symbol].index[trade_table[symbol]['signals'].size-1],inplace=True)
    
    #取出dataframe
    Total=pd.concat(trade_table.values()).reset_index()
    
    #整理交易明細表
    #datelist=sorted(list(set(Total['Date'])))

    buydata = Total.loc[Total['signals'] == 1]
    buydata.columns = ['Buy_Date','Open','High','Low','buyPrice','Adj Close','Volume','symbol','20d','20dstd','UBB','MBB','LBB',
                       'signals','positions']
    buydata=buydata.reset_index()
    buydata.drop(['index','Open','High','Low','Adj Close','20d','20dstd','UBB','MBB','LBB','positions'],axis=1,inplace=True)

    selldata = Total.loc[Total['signals'] == -1]
    selldata.columns = ['Sell_Date','Open','High','Low','sellPrice','Adj Close','Volume','symbol','20d','20dstd','UBB','MBB','LBB',
                        'signals','positions']
    selldata=selldata.reset_index()
    selldata.drop(['index','Open','High','Low','Adj Close','symbol','20d','20dstd','UBB','MBB','LBB'],axis=1,inplace=True)

    result=pd.concat([buydata,selldata], axis=1)
    result=result.sort_values('Buy_Date').reset_index(drop=True)

# 基金化回測

In [ ]:
cash = 10000000
num = 10
account=[]
performance=[]

today_in_fund = pd.DataFrame([],columns=['Buy_Date','buyPrice','Volume','symbol','signals','Sell_Date','sellPrice','Volume',
                                         'signals','positions'])
real_trade = pd.DataFrame([],columns=['Buy_Date','buyPrice','Volume','symbol','signals','Sell_Date','sellPrice','Volume',
                                      'signals','positions'])
stock_hold = pd.DataFrame([],columns=['Date','symbol','Close','positions'])

result['buyPrice']=result['buyPrice']*(1+0.001425)
result['sellPrice']=result['sellPrice']*(1-0.004425)
        

for t in datelist:
    #賣出
    if today_in_fund['Sell_Date'].size >=1:
        today_sell = today_in_fund.loc[today_in_fund['Sell_Date'] == t].reset_index(drop=True)
        #逐筆賣出
        for j in range(0,today_sell['Sell_Date'].size):
            cash = cash + today_sell['sellPrice'][j]*today_sell['positions'][j]*1000
        today_in_fund = today_in_fund.loc[today_in_fund['Sell_Date'] > t].reset_index(drop=True)
    #買入
    today_buy = result.loc[result['Buy_Date'] == t]
    today_buy = today_buy.reset_index(drop=True)
    
    if today_in_fund['Buy_Date'].size < num:
        invest_per_target = (cash/(num-today_in_fund['symbol'].size))
        #逐筆買進
        for i in range(0,today_buy['Buy_Date'].size):
            real_quant = math.floor(invest_per_target/(today_buy['buyPrice'][i]*1000))
            if cash >= real_quant*today_buy['buyPrice'][i]*1000:   #如果現金足夠
                cash = cash-(real_quant*today_buy['buyPrice'][i]*1000)
                today_buy.loc[today_buy.index[i],'positions'] = real_quant
                stock_buy = pd.DataFrame(today_buy.loc[today_buy.index[i],])
                stock_buy = stock_buy.transpose()
                today_in_fund = pd.concat([today_in_fund,stock_buy],axis=0).reset_index(drop=True)
                #紀錄該筆股票有被交易
                
                if today_in_fund.loc[today_in_fund.index[i],'Buy_Date'] == t:
                    real_record = today_in_fund.loc[today_in_fund.index[i],:]
                    real_record = pd.DataFrame(real_record).transpose().reset_index(drop=True)
                    real_record.loc[real_record.index[0],'positions'] = real_quant
                    real_trade = pd.concat([real_trade,real_record],axis=0).reset_index(drop=True)
    #改變表格型態以利製作淨值表
    today_code = list(set(today_in_fund['symbol']))
    today_price = data_df[data_df['Date'] == t ]
    today_price = today_price[today_price.symbol.isin(today_in_fund.symbol)]
    for k in range(0,len(today_code)):
        stock_position = today_in_fund[today_in_fund.symbol.str.contains(today_in_fund.symbol[k])].reset_index(drop=True)
        stock_position = stock_position['positions'].sum()
        stock_price = today_price.loc[today_price['symbol'] == today_in_fund.symbol[k]].reset_index(drop=True)
        stock_price.loc[stock_price.index[0],'positions'] = stock_position
        fund_information = stock_price[['Date','symbol','Close','positions']]
        stock_hold = pd.concat([stock_hold,fund_information],axis=0).reset_index(drop=True)
    
    date = t
    fund_value = stock_hold.loc[stock_hold['Date'] == t]
    fund_value = cash + (fund_value['positions']*fund_value['Close']*1000).sum() #邏輯有問題
    account.append((date,fund_value,cash))
    account_df = pd.DataFrame(account,columns=['Date','Portfolio','Cash'])
    account_df['Return'] = account_df['Portfolio'].pct_change()
    account_df['cumRet'] = account_df['Return'].cumsum().shift()
    account_df['DD'] = np.reciprocal(((np.reciprocal(account_df['Portfolio'])).cummin()/np.reciprocal(account_df['Portfolio'])-1))
    
SharpeRatio = np.sqrt(252.0)*np.mean((account_df['Return']-1.04/252))/np.std(account_df['Return'])
Maxdd = account_df['DD'].min()
vol = np.std(account_df['Return'])
meanRet = np.mean(account_df['Return'])
performance.append((meanRet,vol,SharpeRatio,Maxdd))
performance_df = pd.DataFrame(performance,columns=['MeanReturn','Volatility','Sharpe','MaxmumDrawDown'])

# 實際交易表

In [ ]:
real_trade

# 基金淨值表

In [ ]:
account_df

# 績效評估

In [ ]:
performance_df

# 測試區(不要執行)

In [ ]:
real_buy = real_trade.iloc[:,:5]
real_positions = pd.DataFrame(real_trade.iloc[:,10])

real_buy = pd.concat([real_buy,real_positions],axis=1)
real_sell = real_trade.iloc[:,5:]
real_buy.columns = ['Date','Close','Volume','symbol','signals','positions']
real_sell.columns = ['Date','Close','Volume','symbol','signals','positions']
stock_hold = pd.concat([real_buy,real_sell],axis=0)
stock_hold = stock_hold.sort_values(by='Date').reset_index(drop=True)
if tradetable.loc[tradetable.index[t], 'signals']== 1:
        tradetable.loc[tradetable.index[t], 'Close'] = tradetable.loc[tradetable.index[t], 'Close']*(1+0.001425)
stock_hold

In [ ]:
#測試
cash = 1000000
tradetable['residual_fund'] = 0
tradetable['portfolio'] = 0

for t in range(0,tradetable['symbol'].size):
    if tradetable.loc[tradetable.index[t], 'signals']== 1:
        tradetable.loc[tradetable.index[t], 'Close'] = tradetable.loc[tradetable.index[t], 'Close']*(1+0.001425)
        tradetable.loc[tradetable.index[t], 'positions'] = math.floor(100000/tradetable.loc[tradetable.index[t], 'Close'])
        tradetable.loc[tradetable.index[t], 'residual_fund'] = cash-tradetable.loc[tradetable.index[t], 'Close']*tradetable.loc[tradetable.index[t], 'positions'] 
    if tradetable.loc[tradetable.index[t], 'signals']== -1:
        tradetable.loc[tradetable.index[t], 'Close'] = tradetable.loc[tradetable.index[t], 'Close']*(1-0.004425)
        tradetable.loc[tradetable.index[t], 'positions'] = tradetable.loc[tradetable.index[t-1], 'positions']
        tradetable.loc[tradetable.index[t], 'residual_fund'] = cash+tradetable.loc[tradetable.index[t], 'Close']*tradetable.loc[tradetable.index[t], 'positions']
        

In [ ]:
stock_hold['Cash'] = 0
    stock_hold['portfolio'] = 0
    for a in range(0,stock_hold['symbol'].size):
        if stock_hold.index[a] == 0:
            if stock_hold['signals'][a] == 1:
                stock_hold.loc[stock_hold.index[a], 'Cash'] = 100000000-stock_hold['Close'][a]*stock_hold['positions'][a]*1000
                stock_hold.loc[stock_hold.index[a], 'portfolio'] = 100000000+stock_hold['Close'][a]*stock_hold['positions'][a]*1000
        if stock_hold.index[a] != 0:
            if stock_hold['signals'][a] == 1:
                stock_hold.loc[stock_hold.index[a], 'Cash'] = stock_hold['Cash'][a-1]-stock_hold['Close'][a]*stock_hold['positions'][a]*1000
                stock_hold.loc[stock_hold.index[a], 'portfolio'] = stock_hold['Cash'][a]+stock_hold['Close'][a]*stock_hold['positions'][a]*1000
            if stock_hold['signals'][a] == -1:
                stock_hold.loc[stock_hold.index[a], 'Cash'] = stock_hold['Cash'][a-1]+stock_hold['Close'][a]*stock_hold['positions'][a]*1000
                stock_hold.loc[stock_hold.index[a], 'portfolio'] = stock_hold['portfolio'][a-1]-stock_hold['Close'][a]*stock_hold['positions'][a]*1000

In [ ]:
    #改變表格型態以利製作淨值表
    real_buy = real_trade.iloc[:,:5]
    real_positions = pd.DataFrame(real_trade.iloc[:,10])
    real_buy = pd.concat([real_buy,real_positions],axis=1)
    real_sell = real_trade.iloc[:,5:]
    real_buy.columns = ['Date','Close','Volume','symbol','signals','positions']
    real_sell.columns = ['Date','Close','Volume','symbol','signals','positions']
    stock_hold = pd.concat([real_buy,real_sell],axis=0)
    stock_hold = stock_hold.sort_values(by='Date').reset_index(drop=True)